In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.v2 as transforms_v2  # CutMix를 위한 v2 transforms
import sys
import os
import time
import random
import numpy as np
import wandb
from tqdm import tqdm
from models.resnet import resnet18, resnet34, resnet50
from tools.tool import EarlyStopping  # EarlyStopping 클래스 가져오기

wandb.login(key="ef091b9abcea3186341ddf8995d62bde62d7469e")
wandb.init(project="PBL-2", name="resnet18_cutmix_standard")  # 실험 이름 변경

# WandB 설정
config = {
    "model": "resnet18",
    "batch_size": 128,
    "num_epochs": 100,
    "learning_rate": 0.001,
    "optimizer": "Adam",
    "seed": 2025,
    "deterministic": False,
    "patience": 10,  # early stopping patience
    "cutmix_alpha": 1.0,  # CutMix 알파 파라미터
    "cutmix_prob": 0.5    # CutMix 적용 확률
}
wandb.config.update(config)

# CIFAR-100 데이터셋 로드 - 기본 train/test 분할 사용
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

trainset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=transform_train)

testset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=transform_test)

# DataLoader 생성
trainloader = DataLoader(trainset, batch_size=config["batch_size"], shuffle=True, num_workers=16)
testloader = DataLoader(testset, batch_size=config["batch_size"], shuffle=False, num_workers=16)

print(f"Train set size: {len(trainset)}")
print(f"Test set size: {len(testset)}")

# CutMix 변환 정의
cutmix = transforms_v2.CutMix(alpha=config["cutmix_alpha"], num_classes=100)  # CIFAR-100은 100개 클래스

def train(model, trainloader, criterion, optimizer, device, epoch):
    """
    학습 함수 (CutMix 적용)
    """
    model.train()   # 모델을 학습 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    running_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # CutMix 확률적 적용
        if random.random() < config["cutmix_prob"]:
            inputs, labels = cutmix(inputs, labels)
            # 이 경우 labels은 원-핫 인코딩 형태로 변환됨
            use_cutmix = True
        else:
            use_cutmix = False
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        # CutMix 적용 여부에 따라 손실 함수 선택
        if use_cutmix:
            # CutMix가 적용된 경우 (원-핫 인코딩된 레이블)
            loss = torch.nn.functional.cross_entropy(outputs, labels)
        else:
            # 일반적인 경우 (정수 인덱스 레이블)
            loss = criterion(outputs, labels)
            
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # 정확도 계산 - CutMix 적용 여부에 따라 다르게 처리
        if use_cutmix:
            # 원-핫 인코딩된 레이블에서 argmax를 사용해 가장 큰 값의 인덱스 추출
            _, label_idx = labels.max(1)
        else:
            # 정수 인덱스 레이블 그대로 사용
            label_idx = labels
            
        # top-1 정확도 계산
        _, predicted = outputs.max(1)
        total += inputs.size(0)
        correct_top1 += predicted.eq(label_idx).sum().item()
        
        # top-5 정확도 계산
        _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
        correct_top5 += sum([1 for i in range(len(label_idx)) if label_idx[i] in top5_idx[i]])
        
        if (i + 1) % 50 == 0:  # 50 배치마다 출력
            print(f'Epoch [{epoch+1}], Batch [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}')
    
    epoch_loss = running_loss / len(trainloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    train_time = time.time() - start_time
    
    # 학습 세트에 대한 성능 출력
    print(f'Train set: Epoch: {epoch+1}, Average loss:{epoch_loss:.4f}, LR: {optimizer.param_groups[0]["lr"]:.6f} '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{train_time:.2f}s')
    
    return epoch_loss, accuracy_top1, accuracy_top5

def evaluate(model, dataloader, criterion, device, epoch, phase="test"):
    """
    평가 함수
    """
    model.eval()  # 모델을 평가 모드로 설정
    start_time = time.time()  # 시간 측정 시작
    
    eval_loss = 0.0
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    
    # 그래디언트 계산 비활성화
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 순전파
            outputs = model(inputs)
            
            # 손실 계산
            loss = criterion(outputs, labels)
            eval_loss += loss.item()
            
            # top-1 정확도 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct_top1 += (predicted == labels).sum().item()
            
            # top-5 정확도 계산
            _, top5_idx = outputs.topk(5, 1, largest=True, sorted=True)
            correct_top5 += top5_idx.eq(labels.view(-1, 1).expand_as(top5_idx)).sum().item()
    
    # 평균 손실 및 정확도 계산
    eval_loss = eval_loss / len(dataloader)
    accuracy_top1 = 100.0 * correct_top1 / total
    accuracy_top5 = 100.0 * correct_top5 / total
    
    # 평가 시간 계산
    eval_time = time.time() - start_time
    
    # 테스트 세트에 대한 성능 출력
    print(f'{phase.capitalize()} set: Epoch: {epoch+1}, Average loss:{eval_loss:.4f}, '
          f'Top-1 Accuracy: {accuracy_top1:.4f}%, Top-5 Accuracy: {accuracy_top5:.4f}%, Time consumed:{eval_time:.2f}s')
    print()
    
    return eval_loss, accuracy_top1, accuracy_top5

# EarlyStopping 클래스는 이제 tools.tool에서 가져오기 때문에 제거


# 메인 학습 루프
def main_training_loop(model, trainloader, testloader, criterion, optimizer, device, num_epochs, patience):
    """
    메인 학습 루프 (validation 없이)
    """
    early_stopping = EarlyStopping(patience=patience, verbose=True, path='checkpoint.pt')
    
    best_test_acc_top1 = 0.0
    best_test_acc_top5 = 0.0
    
    # tqdm을 사용한 진행 상황 표시
    for epoch in tqdm(range(num_epochs)):
        # 학습
        train_loss, train_acc_top1, train_acc_top5 = train(model, trainloader, criterion, optimizer, device, epoch)
        
        # 테스트 데이터로 평가
        test_loss, test_acc_top1, test_acc_top5 = evaluate(model, testloader, criterion, device, epoch, phase="test")
        
        # WandB에 로깅
        wandb.log({
            "epoch": epoch + 1,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "train_loss": train_loss,
            "train_accuracy_top1": train_acc_top1,
            "train_accuracy_top5": train_acc_top5,
            "test_loss": test_loss,
            "test_accuracy_top1": test_acc_top1,
            "test_accuracy_top5": test_acc_top5
        })
            
        # 최고 정확도 모델 저장 (top-1 기준)
        if test_acc_top1 > best_test_acc_top1:
            best_test_acc_top1 = test_acc_top1
            best_test_acc_top5_at_best_top1 = test_acc_top5
            print(f'New best top-1 accuracy: {best_test_acc_top1:.2f}%, top-5 accuracy: {best_test_acc_top5_at_best_top1:.2f}%')
            # 모델 저장
            model_path = f'best_model_{wandb.run.name}.pth'
            torch.save(model.state_dict(), model_path)
            
            # WandB에 모델 아티팩트 저장
            wandb.save(model_path)
        
        # top-5 accuracy 기록 업데이트
        if test_acc_top5 > best_test_acc_top5:
            best_test_acc_top5 = test_acc_top5
            print(f'New best top-5 accuracy: {best_test_acc_top5:.2f}%')

        # Early stopping 체크 (test loss 기준)
        early_stopping(test_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered. Training stopped.")
            break
    
    # 훈련 완료 후 최고 모델 로드
    if early_stopping.early_stop:
        print("Loading best model from early stopping checkpoint...")
        model.load_state_dict(torch.load('checkpoint.pt'))
    else:
        print("Loading best model based on test accuracy...")
        model_path = f'best_model_{wandb.run.name}.pth'
        model.load_state_dict(torch.load(model_path))

    # 최종 테스트 세트 평가
    final_test_loss, final_test_acc_top1, final_test_acc_top5 = evaluate(model, testloader, criterion, device, num_epochs-1, phase="test")
    
    print(f'Finish! Best test top-1 accuracy: {best_test_acc_top1:.2f}%, Best test top-5 accuracy: {best_test_acc_top5:.2f}%')
    print(f'Final test top-1 accuracy: {final_test_acc_top1:.2f}%, Final test top-5 accuracy: {final_test_acc_top5:.2f}%')
    
    # WandB에 최종 결과 기록
    wandb.run.summary["best_test_accuracy_top1"] = best_test_acc_top1
    wandb.run.summary["best_test_accuracy_top5"] = best_test_acc_top5
    wandb.run.summary["final_test_accuracy_top1"] = final_test_acc_top1
    wandb.run.summary["final_test_accuracy_top5"] = final_test_acc_top5

    # Early stopping 정보 저장
    if early_stopping.early_stop:
        wandb.run.summary["early_stopped"] = True
        wandb.run.summary["early_stopped_epoch"] = epoch+1
    else:
        wandb.run.summary["early_stopped"] = False


# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 모델 초기화
model = resnet18().to(device)  
criterion = nn.CrossEntropyLoss()  # 손실 함수 정의
optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])  # 옵티마이저 정의

# WandB에 모델 구조 기록
wandb.watch(model, log="all")

# GPU 가속
if torch.cuda.device_count() > 1:
    print(f"{torch.cuda.device_count()}개의 GPU를 사용합니다.")
    model = nn.DataParallel(model)

# 훈련 시작 시간 기록
start_time = time.time()

# 메인 학습 루프 호출
main_training_loop(
    model=model,
    trainloader=trainloader,
    testloader=testloader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    num_epochs=config["num_epochs"],
    patience=config["patience"]
)

# 훈련 종료 시간 기록 및 출력
end_time = time.time()
total_time = end_time - start_time
wandb.log({"total_training_time": total_time})

print(f"Total training time: {total_time:.2f} seconds")

# WandB 실행 종료
wandb.finish()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/guswls/.netrc


Files already downloaded and verified
Files already downloaded and verified
Train set size: 50000
Test set size: 10000
Using device: cuda
2개의 GPU를 사용합니다.


  0%|                                                                                                       | 0/100 [00:00<?, ?it/s]

Epoch [1], Batch [50/391], Loss: 4.2820
Epoch [1], Batch [100/391], Loss: 4.0357
Epoch [1], Batch [150/391], Loss: 4.2454
Epoch [1], Batch [200/391], Loss: 4.2672
Epoch [1], Batch [250/391], Loss: 4.1416
Epoch [1], Batch [300/391], Loss: 3.4667
Epoch [1], Batch [350/391], Loss: 4.3060
Train set: Epoch: 1, Average loss:3.9793, LR: 0.001000 Top-1 Accuracy: 10.4180%, Top-5 Accuracy: 30.2480%, Time consumed:50.66s
Test set: Epoch: 1, Average loss:3.3190, Top-1 Accuracy: 19.1700%, Top-5 Accuracy: 48.0700%, Time consumed:7.94s

New best top-1 accuracy: 19.17%, top-5 accuracy: 48.07%
New best top-5 accuracy: 48.07%
Validation loss decreased (inf --> 3.318960). Saving model ...


  1%|▉                                                                                            | 1/100 [00:58<1:37:02, 58.81s/it]

...

Test set: Epoch: 21, Average loss:1.6134, Top-1 Accuracy: 59.3900%, Top-5 Accuracy: 83.4700%, Time consumed:9.36s

EarlyStopping 카운터: 9 / 10
Epoch [22], Batch [50/391], Loss: 2.4643
Epoch [22], Batch [100/391], Loss: 2.0788
Epoch [22], Batch [150/391], Loss: 1.2459
Epoch [22], Batch [200/391], Loss: 1.6774
Epoch [22], Batch [250/391], Loss: 0.0786
Epoch [22], Batch [300/391], Loss: 2.6005
Epoch [22], Batch [350/391], Loss: 0.0645
Train set: Epoch: 22, Average loss:0.9922, LR: 0.001000 Top-1 Accuracy: 86.6860%, Top-5 Accuracy: 96.0740%, Time consumed:48.61s


 21%|███████████████████▎                                                                        | 21/100 [21:18<1:20:08, 60.87s/it]

Test set: Epoch: 22, Average loss:1.5816, Top-1 Accuracy: 59.9100%, Top-5 Accuracy: 83.7300%, Time consumed:8.83s

EarlyStopping 카운터: 10 / 10
Early stopping triggered. Training stopped.
Loading best model from early stopping checkpoint...


Test set: Epoch: 100, Average loss:1.5154, Top-1 Accuracy: 59.1100%, Top-5 Accuracy: 85.1900%, Time consumed:8.45s

Finish! Best test top-1 accuracy: 60.20%, Best test top-5 accuracy: 85.19%
Final test top-1 accuracy: 59.11%, Final test top-5 accuracy: 85.19%
Total training time: 1286.77 seconds


epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_accuracy_top1,▁▃▄▆▆▆▇▇██████████████
test_accuracy_top5,▁▄▅▇▇▇████████████████
test_loss,█▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_training_time,▁
train_accuracy_top1,▁▂▃▃▄▅▅▅▆▆▆▇▇█▇███████
train_accuracy_top5,▁▃▄▅▆▆▆▇▇▇▇▇██████████
train_loss,█▇▆▅▄▄▄▃▃▂▂▂▂▁▂▁▁▁▁▁▁▁
best_test_accuracy_top1,60.2
best_test_accuracy_top5,85.19
